# Terraform Enterprise Install on AWS - Stage 2

* Stage 1: **TFE Standalone Prerequisites on AWS**
    * https://github.com/hashicorp-services/terraform-aws-tfe-prereqs
* Stage 2: **TFE Standalone on AWS - Quick Install**
    * https://github.com/hashicorp-services/terraform-aws-tfe
* Motivated by **Automated Installation of PTFE with External Services in AWS**.
	* Source: https://github.com/hashicorp/private-terraform-enterprise/tree/automated-aws-pes-installation

## Overview

This is a guide for deploying Terraform Enterprise on AWS using HashiCorp’s IS Terraform module. It will perform an [automated installations](https://www.terraform.io/docs/enterprise/private/automating-the-installer.html) of [Terraform Enterprise](https://www.terraform.io/docs/enterprise/private/index.html).

> **NOTE:** The repo is close to, but not quite production-ready. It **should NOT** be used for an enterprise customer production deployment without some tweaking/customization/hardening.

This module supports a number of different configurations/scenarios:


* **OS:** Ubuntu or Amazon Linux.
* **Operational Mode**: `External Services`
* **Installation Methods**: `online` and `airgap`
* Active/Active
* Supports public and private networks.

See the [examples](https://github.com/hashicorp/is-terraform-aws-tfe-standalone-quick-install/blob/master/examples) section for detailed deployment scenarios & instructions. 

Several assumptions are made and default values are set for some of the resource arguments to reduce complexity and the amount of required inputs (see the [Security Caveats](#Security-Caveats)  section).

[Guide to is-terraform-aws-tfe-standalone-quick-install - Required Inputs](bear://x-callback-url/open-note?id=BE232B3D-94AE-49B6-9B33-B62095551C8C-66557-000513BBCB51AF13&header=Required%20Inputs)
[Guide to is-terraform-aws-tfe-standalone-quick-install - Optional Inputs](bear://x-callback-url/open-note?id=BE232B3D-94AE-49B6-9B33-B62095551C8C-66557-000513BBCB51AF13&header=Optional%20Inputs)
[Guide to is-terraform-aws-tfe-standalone-quick-install - Outputs](bear://x-callback-url/open-note?id=BE232B3D-94AE-49B6-9B33-B62095551C8C-66557-000513BBCB51AF13&header=Outputs)

## Prerequisites

- Terraform >= 0.14 installed on clients/workstations
- TFE license file from Replicated (named `tfe-license.rli`). Stored in Terraform working directory
- AWS account w/ the ability to deploy resources via Terraform CLI
- S3 bucket for [S3 Remote State backend](https://www.terraform.io/docs/language/settings/backends/s3.html) is recommended
- S3 "`bootstrap`" bucket used to stage required files to automate TFE install:
  - TFE license file from Replicated (`.rli` file extension)
  - TFE airgap bundle (only if installation method is _airgap_)
  - `replicated.tar.gz` bundle (only if installation method is _airgap_)
- VPC w/ subnets (minimum of 2) in different Availability Zones
- TLS/SSL certificate in one of the following configurations:
  - Route53 Hosted Zone of the type **public** so ACM certificate validation works
  - Custom CA certificate files staged in AWS Secrets Manager
      - Common Name needs to match the desired TFE hostname/FQDN (`tfe_hostname` required input)
- TFE install secrets stored in AWS Secrets Manager or specified directly as input variables
- A mechanism for shell access to EC2 instance (SSH key pair, SSM, etc.)  

> Note: a prereqs helper module exists [here](https://github.com/hashicorp/is-terraform-aws-tfe-standalone-prereqs).

## Getting Started

A good place to start is with one of the [test deployment scenarios](https://github.com/hashicorp-services/terraform-aws-tfe/blob/main/tests/README.md). 
* The _deployment scenarios_ contain actual Terraform code that can be referenced and deployed after populating a `terraform.tfvars` file.
* The "happy path" deployment scenario that is probably the quickest and easiest to deploy and manage can be found [here](https://github.com/hashicorp-services/terraform-aws-tfe/blob/main/tests/alb-ext-r53-acm-ol/README.md). The input variable default values of the module favor this scenario.
* Alternatively, the more locked-down-network/security option can be found [here](https://github.com/hashicorp-services/terraform-aws-tfe/blob/main/tests/nlb-int-r53-cust-ag/README.md). (for air-gapped deployements)

For this tutorial, we will take the happy path. You can jump there now.

## Usage
The following subsections below contain details on features, modes, and settings that this module supports - and how to configure them.

### Installation Method
Both **online** and **airgap** installation methods are supported by this module.

* For **online**, specify a value for `tfe_release_sequence`.
    * Omit `airgap_install`, `replicated_bundle_path`, and `tfe_airgap_bundle_path`.<br>
    For example:
```go
tfe_release_sequence = 568
```
* For **airgap**, specify values for `airgap_install`, `replicated_bundle_path`, and `tfe_airgap_bundle_path`.
    * Omit `tfe_release_sequence`.<br>
    For example:
```go
airgap_install         = true
replicated_bundle_path = "s3://my-tfe-bootstrap-bucket/replicated.tar.gz"
tfe_airgap_bundle_path = "s3://my-tfe-bootstrap-bucket/tfe-568.airgap"
```

**NOTE:** If `airgap_install = true`, the `user_data` (cloud-init) script assumes Linux package repositories cannot be reached. So, the AMI specified must contain the following software dependencies:
- `unzip`
- `jq`
- `awscli`
- `docker` (version 19.03.8 recommended)

> **NOTE:** If you are deploying into an airgapped environment but your EC2 instances can reach a Linux package repository, then modify the `install_dependencies()` function within the `./templates/tfe_user_data.sh.tpl` file.
<p>&nbsp;</p>

### Load Balancing

This module supports both the AWS Application Load Balancer (ALB) and the Network Load Balancer (NLB) via the input variable `load_balancer_type`. The [test deployment scenarios](./tests/README.md) section contains more detail on how to choose based on the _deployment_scenario_, but in general here are the differences:

#### ALB
- Module default variable values favor the ALB deployment scenarios
- ALB is a layer 7 HTTPS load balancer and works really well when used in combination with Route53 and ACM
- Module will either create the TFE TLS/SSL certificate on the fly via ACM, or the certificate can be imported into ACM as a module prereq
- Input variable `tls_bootstrap_type` defaults to `self-signed` because the TFE TLS/SSL certificate will be terminated at the load balancer-level

#### NLB
- NLB is a layer 4 TCP load balancer that does TLS-passthrough in the supported deployment scenarios
- NLB is ideal for users who want to terminate TLS/SSL end-to-end at the instance-level rather than at the load balancer-level
- Input variable `tls_bootstrap_type` should be set to `server-path` because the TFE server certificate files must exist on the instance where TLS/SSL will be terminated
- Typically the TFE server certificate files are placed in AWS Secrets Manager as a module prereq (see [TLS/SSL-Certificates](####TLS/SSL-Certificates) within the [Secrets Management](###Secrets-Management) section)
- The subnet CIDR's must be in the `ingress_cidr_443_allow` so the NLB health checks can reach the instance(s) properly
- If the NLB is deployed onto private subnets and the `load_balancer_scheme` is set to `internal`, then `hairpin_addressing` must be set to `true` because the NLB does not support loopback addressing
- If `hairpin_addressing` is `true` and a proxy is also in use, then the TFE FQDN (`tfe_hostname`) must be added to the `extra_no_proxy` input
<p>&nbsp;</p>

### Secrets Management

By default, this module expects the necessary secrets for the deployment to be stored in AWS Secrets Manager as a module prereq. Terraform will automatically grant the TFE IAM Instance Profile read access to the secrets based on the ARNs specified for the corresponding input variables detailed below. Then, when the EC2 instance boots and the cloud-init process executes, the secrets will automatically be retrieved from Secrets Manager via `awscli`.

#### Install Secrets
Create a single secret with two key/value pairs, naming the keys exactly as follows:
  - `console_password`
  - `enc_password`

Specify the ARN of the secret for the input variable `tfe_install_secrets_arn`. If it is not desirable to use AWS Secrets Manager for the _install secrets_, it is also possible to specify the secrets directly as variable values for the input variables `console_password` and `enc_password`.

#### TLS/SSL Certificates (optional)
These secrets are only necessary in deployment scenarios where `tls_bootstrap_type` is set to `server-path` (and typically the NLB is in use).

- **Certificate** - store this as a plaintext secret in Secrets Manager and specify the ARN for the input variable `tfe_cert_secret_arn`
- **Private Key** - store this as a base64-encoded plaintext secret in Secrets Manager and specify the ARN for the input variable `tfe_privkey_secret_arn`\
(_i.e._ `cat privkey.pem | base64 -w 0`)
- **Custom CA bundle** - store this as a plaintext secret, replace any new lines with `\n` because JSON does not support raw new line characters, and specify the ARN for the input variable `ca_bundle_secret_arn`

An example command to help format the string before storing your custom CA bundle as a secret for `ca_bundle_secret_arn`:
```shell
sed -e ':a' -e 'N' -e '$!ba' -e 's/\n/\\n/g' ./my-custom-ca-bundle.pem
```
<p>&nbsp;</p>

### Active-Active
This module supports deploying TFE in the Active/Active architecture, including external Redis and multiple nodes within the Autoscaling Group. For more detailed instructions on the upgrade path to Active/Active, see the [Active/Active](./docs/active-active.md) page. In order to enable Active/Active, specify the following input variables:
```go
enable_active_active = true
redis_subnet_ids     = ["subnet-00000000000000000", "subnet-11111111111111111", "subnet-22222222222222222"] # private subnet IDs
redis_password       = "MyTfeRedisPasswd123!" # optional password to enable transit encryption with Redis
```
<p>&nbsp;</p>

### Database
This module supports an Amazon RDS and AWS Aurora RDS for the database requirement of TFE. The variable `rds_is_aurora` controls which database type is created. By default this is set to `false` and an AWS RDS database will be deployed. This module supports the initial deployment of either database type, however it does not support migration between one or the other after deployment. If this variable is changed after TFE has been deployed the existing database will be deleted and a new one created, which will result in data loss.
<p>&nbsp;</p>

### Alternative Terraform Build Worker (TBW) Image
The module supports running a custom Terraform Build Worker container image. The only supported container registry at this time (without requiring additional tweaks to the module code) is AWS Elastic Container Registry (ECR). In order to enable this functionality, specify the following input variables:
```go
tbw_image            = "custom_image"
custom_tbw_ecr_repo  = "<my-ecr-repo-name>"
custom_tbw_image_tag = "<my-image-tag>" (e.g. `v1` or `latest`)
```

Pushing the docker image up to the ECR repository becomes an additional module prereq when leveraging this functionality.
<p>&nbsp;</p>

### Log Forwarding
Currently by default, this module supports configuring AWS Cloud Watch as a destination for the [TFE Log Forwarding](https://www.terraform.io/docs/enterprise/admin/logging.html) feature. To enable this functionality, specify the following input variables:
```go
log_forwarding_enabled    = true
cloudwatch_log_group_name = "<my-tfe-log-group>"
```
<p>&nbsp;</p>

## Post Deploy

## Troubleshooting

### Terraform Apply

* I ran into issues with the user-data file being too big (>16KB).
    * I added this resource to the TFE module to see what the user-data file looks like.
    * Determined that copying the license file from a local file did not work. I used s3 instead.

In [ ]:
cat > config/terraform/terraform-aws-tfe/local_file.tf <<"EOF"
resource "local_file" "inventory" {
  content = templatefile("${path.module}/templates/tfe_user_data.sh.tpl", local.user_data_args )
  filename = "cloud_init_deleteme"
}
EOF

### Network Issues

* By default, the ALB security group only allows access to port 880 to the 10.0.0.0/16 subnet.
    * You can add your IP or another subnet to the allow list.
    * https://us-west-2.console.aws.amazon.com/ec2/v2/home?region=us-west-2#LoadBalancers:search=pphan;sort=loadBalancerName

---

## Day 2 Operations
See the [operations](./docs/operations.md) page for more information on TFE administrative tasks and procedures.
<p>&nbsp;</p>

## Providers

| Name | Version |
|------|---------|
| aws | `>= 3.33.0` |
| random | `3.1.0` |
| template | `2.2.0` |

## Inputs

| Name | Description | Type | Default | Required |
|------|-------------|------|---------|:--------:|
| airgap\_install | Boolean for TFE installation method to be airgap. | `bool` | `false` | no |
| ami\_id | Custom AMI ID for TFE EC2 Launch Template. If specified, value of `os_distro` must coincide with this custom AMI OS distro. | `string` | `null` | no |
| asg\_health\_check\_grace\_period | The amount of time to wait for a new TFE instance to be healthy. If this threshold is breached, the ASG will terminate the instance and launch a new one. | `number` | `900` | no |
| asg\_instance\_count | Desired number of EC2 instances to run in Autoscaling Group. Leave at `1` unless Active/Active is enabled. | `number` | `1` | no |
| asg\_max\_size | Max number of EC2 instances to run in Autoscaling Group. Increase after Active/Active is enabled. | `number` | `1` | no |
| aurora\_rds\_availability\_zones | List of Availability Zones to spread Aurora DB cluster across. | `list(string)` | `null` | no |
| aurora\_rds\_engine\_mode | Aurora engine mode. | `string` | `"provisioned"` | no |
| aurora\_rds\_engine\_version | Engine version of Aurora PostgreSQL. | `number` | `12.4` | no |
| aurora\_rds\_global\_cluster\_id | Aurora Global Database cluster identifier. Intended to be used by Aurora DB Cluster instance in Secondary region. | `string` | `null` | no |
| aurora\_rds\_instance\_class | Instance class of Aurora PostgreSQL database. | `string` | `"db.r5.xlarge"` | no |
| aurora\_rds\_replica\_count | Amount of Aurora Replica instances to deploy within the Aurora DB cluster within the same region. | `number` | `1` | no |
| aurora\_rds\_replication\_source\_identifier | ARN of a source DB cluster or DB instance if this DB cluster is to be created as a Read Replica. Intended to be used by Aurora Replica in Secondary region. | `string` | `null` | no |
| aurora\_source\_region | Source region for Aurora Cross-Region Replication. Only specify for Secondary instance. | `string` | `null` | no |
| aws\_ssm\_enable | Boolean to attach the `AmazonSSMManagedInstanceCore` policy to the TFE role, allowing the SSM agent (if present) to function. | `bool` | `false` | no |
| bucket\_replication\_configuration | Map containing S3 Cross-Region Replication configuration. | `any` | `{}` | no |
| ca\_bundle\_secret\_arn | ARN of AWS Secrets Manager secret for private/custom CA bundles. New lines must be replaced by `<br>` character prior to storing as a plaintext secret. | `string` | `""` | no |
| capacity\_concurrency | Total concurrent Terraform Runs (Plans/Applies) allowed within TFE. | `string` | `"10"` | no |
| capacity\_memory | Maxium amount of memory (MB) that a Terraform Run (Plan/Apply) can consume within TFE. | `string` | `"512"` | no |
| cloudwatch\_log\_group\_name | Name of CloudWatch Log Group to configure as log forwarding destination. `log_forwarding_enabled` must also be `true`. | `string` | `""` | no |
| common\_tags | Map of common tags for taggable AWS resources. | `map(string)` | `{}` | no |
| console\_password | Password to unlock TFE Admin Console accessible via port 8800. Specify `aws_secretsmanager` to retrieve from AWS Secrets Manager via `tfe_install_secrets_arn` input. | `string` | `"aws_secretsmanager"` | no |
| create\_tfe\_alias\_record | Boolean to create Route53 Alias Record for `tfe_hostname` resolving to Load Balancer DNS name. If `true`, `route53_hosted_zone_tfe` is also required. | `bool` | `true` | no |
| custom\_tbw\_ecr\_repo | Name of AWS Elastic Container Registry (ECR) Repository where custom Terraform Build Worker (tbw) image exists. Only specify if `tbw_image` is set to `custom_image`. | `string` | `""` | no |
| custom\_tbw\_image\_tag | Tag of custom Terraform Build Worker (tbw) image. Examples: `v1`, `latest`. Only specify if `tbw_image` is set to `custom_image`. | `string` | `"latest"` | no |
| destination\_bucket | Destination S3 Bucket for Cross-Region Replication configuration. Should exist in Secondary region. | `string` | `""` | no |
| ec2\_subnet\_ids | List of subnet IDs to use for the EC2 instance. Private subnets is the best practice. | `list(string)` | n/a | yes |
| enable\_active\_active | Boolean to enable TFE Active/Active and in turn deploy Redis cluster. | `bool` | `false` | no |
| enable\_metrics\_collection | Boolean to enable internal TFE metrics collection. | `bool` | `true` | no |
| enc\_password | Password to protect unseal key and root token of TFE embedded Vault. Specify `aws_secretsmanager` to retrieve from AWS Secrets Manager via `tfe_install_secrets_arn` input. | `string` | `"aws_secretsmanager"` | no |
| encrypt\_ebs | Boolean for encrypting the root block device of the TFE EC2 instance(s). | `bool` | `false` | no |
| extra\_no\_proxy | A comma-separated string of hostnames or IP addresses to add to the TFE no\_proxy list. Only specify if a value for `http_proxy` is also specified. | `string` | `""` | no |
| force\_tls | Boolean to require all internal TFE application traffic to use HTTPS by sending a 'Strict-Transport-Security' header value in responses, and marking cookies as secure. Only enable if `tls_bootstrap_type` is `server-path`. | `bool` | `false` | no |
| friendly\_name\_prefix | String value for friendly name prefix for AWS resource names. | `string` | n/a | yes |
| hairpin\_addressing | Boolean to enable TFE services to direct requests to the servers' internal IP address rather than the TFE hostname/FQDN. Only enable if `tls_bootstrap_type` is `server-path`. | `bool` | `false` | no |
| http\_proxy | Proxy address to configure for TFE to use for outbound connections/requests. | `string` | `""` | no |
| ingress\_cidr\_22\_allow | List of CIDR ranges to allow SSH ingress to TFE EC2 instance (i.e. bastion host IP, workstation IP, etc.). | `list(string)` | `[]` | no |
| ingress\_cidr\_443\_allow | List of CIDR ranges to allow ingress traffic on port 443 to TFE server or load balancer. | `list(string)` | <pre>[<br>  "0.0.0.0/0"<br>]</pre> | no |
| ingress\_cidr\_8800\_allow | List of CIDR ranges to allow TFE Replicated admin console (port 8800) traffic ingress to TFE server or load balancer. | `list(string)` | `null` | no |
| instance\_size | EC2 instance type for TFE Launch Template. | `string` | `"m5.xlarge"` | no |
| is\_secondary | Boolean indicating whether TFE instance deployment is for Primary region or Secondary region. | `bool` | `false` | no |
| kms\_key\_arn | ARN of KMS key to encrypt TFE RDS, S3, EBS, and Redis resources. | `string` | `""` | no |
| lb\_subnet\_ids | List of subnet IDs to use for the load balancer. If LB is external, these should be public subnets. | `list(string)` | n/a | yes |
| load\_balancer\_scheme | Load balancer exposure. Specify `external` if load balancer is to be public/external-facing, or `internal` if load balancer is to be private/internal-facing. | `string` | `"external"` | no |
| load\_balancer\_type | String indicating whether the load balancer deployed is an Application Load Balancer (alb) or Network Load Balancer (nlb). | `string` | `"alb"` | no |
| log\_forwarding\_enabled | Boolean to enable TFE log forwarding at the application level. | `bool` | `false` | no |
| os\_distro | Linux OS distribution for TFE EC2 instance. Choose from `amzn2`, `ubuntu`, `rhel`, `centos`. | `string` | `"amzn2"` | no |
| rds\_allocated\_storage | Size capacity (GB) of RDS PostgreSQL database. | `string` | `"50"` | no |
| rds\_allow\_major\_version\_upgrade | Boolean to allow major version upgrades of the database. | `bool` | `false` | no |
| rds\_auto\_minor\_version\_upgrade | Boolean to enable the automatic upgrading of new minor versions during the specified `rds_preferred_maintenance_window`. | `bool` | `true` | no |
| rds\_backup\_retention\_period | The number of days to retain backups for. Must be between 0 and 35. Must be greater than 0 if the database is used as a source for a Read Replica. | `number` | `35` | no |
| rds\_copy\_tags\_to\_snapshot | Boolean to enable copying tags to RDS snapshot. | `bool` | `true` | no |
| rds\_database\_name | Name of database. | `string` | `"tfe"` | no |
| rds\_deletion\_protection | Boolean to proctect the database from being deleted. The database cannot be deleted when `true`. | `bool` | `false` | no |
| rds\_engine\_version | Version of RDS PostgreSQL. | `number` | `12.7` | no |
| rds\_instance\_class | Instance class of RDS PostgreSQL database. | `string` | `"db.m5.xlarge"` | no |
| rds\_is\_aurora | Boolean for deploying global Amazon Aurora PostgreSQL instead of Amazon RDS for PostgreSQL | `bool` | `false` | no |
| rds\_multi\_az | Boolean to create a standby instance in a different AZ than the primary and enable HA. | `bool` | `true` | no |
| rds\_password | Password for RDS master DB user. | `string` | n/a | yes |
| rds\_preferred\_backup\_window | Daily time range (UTC) for RDS backup to occur. Must not overlap with `rds_preferred_maintenance_window` if specified. | `string` | `"04:00-04:30"` | no |
| rds\_preferred\_maintenance\_window | Window (UTC) to perform RDS database maintenance. Must not overlap with `rds_preferred_backup_window` if specified. | `string` | `"Sun:08:00-Sun:09:00"` | no |
| rds\_skip\_final\_snapshot | Boolean for RDS to take a final snapshot. | `bool` | `false` | no |
| rds\_subnet\_ids | List of subnet IDs to use for RDS Database Subnet Group. Private subnets is the best practice. | `list(string)` | n/a | yes |
| rds\_username | Username for the master DB user. | `string` | `"tfe"` | no |
| redis\_at\_rest\_encryption\_enabled | Boolean to enable encryption at rest on Redis cluster. A `kms_key_arn` is required when set to `true`. | `bool` | `false` | no |
| redis\_engine\_version | Redis version number | `string` | `"5.0.6"` | no |
| redis\_multi\_az\_enabled | Boolean for deploying Redis nodes in multiple Availability Zones and enabling automatic failover. | `bool` | `true` | no |
| redis\_node\_type | Type of Redis node from a compute, memory, and network throughput standpoint. | `string` | `"cache.m4.large"` | no |
| redis\_parameter\_group\_name | Name of parameter group to associate with Redis cluster. | `string` | `"default.redis5.0"` | no |
| redis\_password | Password (auth token) used to enable transit encryption (TLS) with Redis. | `string` | `""` | no |
| redis\_port | Port number the Redis nodes will accept connections on. | `number` | `6379` | no |
| redis\_replica\_count | Number of replica nodes in Redis cluster. | `number` | `1` | no |
| redis\_subnet\_ids | List of subnet IDs to use for Redis cluster subnet group. | `list(string)` | `null` | no |
| remove\_import\_settings\_from | Replicated setting to automatically remove the `/etc/tfe-settings.json` file (referred to as `ImportSettingsFrom` by Replicated) after installation. | `bool` | `false` | no |
| replicated\_bundle\_path | Full path of Replicated bundle (`replicated.tar.gz`) in S3 bucket. A local filepath is not supported because the Replicated bundle is too large for user\_data. Only specify if `airgap_install` is `true`. Should start with `s3://`. | `string` | `""` | no |
| restrict\_worker\_metadata\_access | Boolean to block Terraform build worker containers from being able to access the EC2 instance metadata endpoint. | `bool` | `false` | no |
| route53\_hosted\_zone\_acm | Route53 Hosted Zone name to create ACM Certificate Validation CNAME record in. Required if `tls_certificate_arn` is not specified. | `string` | `null` | no |
| route53\_hosted\_zone\_tfe | Route53 Hosted Zone name to create `tfe_hostname` Alias record in. Required if `create_tfe_alias_record` is `true`. | `string` | `null` | no |
| ssh\_key\_pair | Name of existing SSH key pair to attach to TFE EC2 instance. | `string` | `""` | no |
| syslog\_endpoint | Syslog endpoint for Logspout to forward TFE logs to. | `string` | `""` | no |
| tbw\_image | Terraform Build Worker container image to use. Set this to `custom_image` to use alternative container image. | `string` | `"default_image"` | no |
| tfe\_airgap\_bundle\_path | Full path of TFE airgap bundle in S3 bucket. A local filepath is not supported because the airgap bundle is too large for user\_data. Only specify if `airgap_install` is `true`. Should start with `s3://`. | `string` | `""` | no |
| tfe\_bootstrap\_bucket | Name of existing S3 bucket containing prerequisite files for TFE automated install. Typically would contain TFE license file and airgap files if `airgap_install` is `true`. | `string` | `""` | no |
| tfe\_cert\_secret\_arn | ARN of AWS Secrets Manager secret for TFE server certificate in PEM format. Required if `tls_bootstrap_type` is `server-path`; otherwise ignored. | `string` | `""` | no |
| tfe\_hosted\_zone\_is\_private | Boolean indicating if `route53_hosted_zone_tfe` is a private zone. | `bool` | `false` | no |
| tfe\_hostname | Hostname/FQDN of TFE instance. This name should resolve to the load balancer DNS name and will be how users and systems access TFE. | `string` | n/a | yes |
| tfe\_install\_secrets\_arn | ARN of AWS Secrets Manager secret metadata for TFE install secrets. If specified, secret must contain key/value pairs for `console_password`, and `enc_password` | `string` | `""` | no |
| tfe\_license\_filepath | Full filepath of TFE license file (`.rli` file extension). A local filepath or S3 is supported. If s3, the path should start with `s3://`. | `string` | n/a | yes |
| tfe\_privkey\_secret\_arn | ARN of AWS Secrets Manager secret for TFE private key in PEM format and base64 encoded. Required if `tls_bootstrap_type` is `server-path`; otherwise ignored. | `string` | `""` | no |
| tfe\_release\_sequence | TFE application version release sequence number within Replicated. Ignored if `airgap_install` is `true`. | `number` | `0` | no |
| tfe\_tls\_certificate\_arn | ARN of TFE certificate imported in ACM to be used for Application Load Balancer HTTPS listeners. Required if `route53_hosted_zone_acm` is not specified. | `string` | `null` | no |
| tls\_bootstrap\_type | Defines where to terminate TLS/SSL. Set to `self-signed` to terminate at the load balancer, or `server-path` to terminate at the instance-level. | `string` | `"self-signed"` | no |
| vpc\_id | VPC ID that TFE will be deployed into. | `string` | n/a | yes |

## Outputs

| Name | Description |
|------|-------------|
| admin\_console\_url | URL of TFE (Replicated) Admin Console based on `tfe_hostname` input. |
| aurora\_aws\_rds\_cluster\_endpoint | Aurora DB cluster instance endpoint. |
| aurora\_rds\_cluster\_arn | ARN of Aurora DB cluster. |
| aurora\_rds\_cluster\_members | List of instances that are part of this Aurora DB Cluster. |
| aurora\_rds\_global\_cluster\_id | Aurora Global Database cluster identifier. |
| aws\_db\_instance\_arn | ARN of RDS DB instance. |
| aws\_db\_instance\_endpoint | RDS DB instance endpoint. |
| lb\_dns\_name | DNS name of the Load Balancer. |
| s3\_bucket\_arn | ARN of TFE S3 bucket. |
| s3\_bucket\_name | Name of TFE S3 bucket. |
| s3\_crr\_iam\_role\_arn | ARN of S3 Cross-Region Replication IAM Role. |
| url | URL of TFE application based on `tfe_hostname` input. |

## Explanation of the Two Stage Deployment Model

We deploy the *AWS infrastructure* and TFE in two stages using the open source flavor of Terraform.

### Stage 1
* Deploy **network** and **security group** resources that the EC2 instances will run in
* Deploy a private **S3 bucket** (source) to upload the PTFE software, license, and settings files
* Deploy a **KMS key** to encrypt the S3 source bucket.
* If deploying a private network, then deploy an EC2 instance as a **bastion host**.

### Stage 2
* Deploy the external PostgreSQL database and S3 bucket (runtime) used in the [Production - External Services](https://www.terraform.io/docs/enterprise/private/preflight-installer.html#operational-mode-decision) operational mode of TFE.
* Deploy the **Auto Scaling Group** and **Launch Template** that will run TFE. 
* Deploy an **Application Load Balancer** and associated resources
* Deploy some required **IAM** resources.

> NOTE: We are creating an S3 bucket in both stages
>	* The bucket created in **Stage 1** is the "**TFE source bucket**"
>	* The bucket created in **Stage 2** is the "**TFE runtime bucket**".

### Why Two Stages?
There are two reasons for splitting the deployment into two stages:
* **Main reason**: Some users are not allowed to provision their own VPC, subnets, and security groups. They would skip Stage 1. 
	* Users who are allowed to provision all required AWS resources, will perform Stage 1.
	* Deploy the network and security group resources and the TFE source bucket from the [network](https://github.com/hashicorp/private-terraform-enterprise/blob/automated-aws-pes-installation/examples/aws/network) directory
	* Copy the IDs of the VPC, subnets, and security groups into a **terraform.tfvars** file in the [aws](https://github.com/hashicorp/private-terraform-enterprise/blob/automated-aws-pes-installation/examples/aws) directory
	* Then deploy the rest of the resources in Stage 2.
* **Second reason**: Some users want to be able to "repave" their TFE instances periodically. For example, they want to destroy the instances and recreate them (possibly with a new AMI).
	* These users need to create the TFE **source bucket** and then place the TFE software and license file in it before they run the Terraform code in **Stage 2**.

---

### Stage 1 Prerequisites
* You need to have an AWS account before running the **Stage 1** Terraform code in either the [network-public](https://github.com/hashicorp/private-terraform-enterprise/blob/automated-aws-pes-installation/examples/aws/network-public) or the [network-private](https://github.com/hashicorp/private-terraform-enterprise/blob/automated-aws-pes-installation/examples/aws/network-private) directory of this repository.

NOTE: A [prereqs helper](https://github.com/hashicorp/is-terraform-aws-tfe-standalone-prereqs) module is available for reference and/or use.

### Stage 2 Prerequisites
You need to have the following things before running the **Stage 2** Terraform code in the [aws](https://github.com/hashicorp/private-terraform-enterprise/blob/automated-aws-pes-installation/examples/aws) directory of this repository:
* **VPC** (provisioned in Stage 1)
* At least two **Subnets** in the VPC (provisioned in Stage 1)
	* You can use the same subnets or use separate subnets for the following:
		* the EC2 instances, the PostgreSQL database, and the ALB
	* If you have more than two subnets, make sure that the two specified in the **alb_subnet_ids** variable include those containing the EC2 instances running PTFE.
* **Security Group** - provisioned in **Stage 1**
* **S3 Bucket** - provisioned in **Stage 1** - to be used as the TFE source bucket
* AWS **KMS key** - provisioned in **Stage 1**
* AWS **AMI** running a Ubuntu or Amazon Linux in the region in which you plan to deploy TFE
* AWS **key pair** that can be used to SSH to the EC2 instances that will be provisioned to run TFE.
* AWS **Route 53 zone** to host the Route 53 record set that will be provisioned.
	* This needs to be a public zone so that the ACM cert created can be validated against a record set created in that zone.
	* If you absolutely need to use a private zone, provide your own ACM cert and remove the code that provisions the additional ACM cert.
* **TFE license**
* If doing an airgap install, need a TFE airgap bundle and **replicated.tar.gz** installer bootstrapper that you can upload to the TFE **source bucket**.
	* Access to Docker and packages it requires so that you can upload them to the TFE source bucket.
* You can also provide the ARN of a certificate that you uploaded into or created within Amazon Certificate Manager (ACM).
    * This will be attached to the listeners created for the application load balancer that will be provisioned in front of the EC2 instances.
    * The **Stage 2** Terraform code actually creates an ACM certificate whether you provide one or not, but if you do provide your own, the generated one is associated with a fake domain consisting of "`fake-`" concatenated to your hostname.
    * If you set the `ssl_certificate_arn` variable to `""`, the generated ACM cert will be associated with your hostname.
    * We generate an ACM cert even if you provide your own in order to make the generation of an ACM cert optional in Terraform 0.11. (This will not be needed with Terraform 0.12.)


---

## Usage

Follow these steps to deploy TFE in your AWS account.

## Provision Stage 1

* For the purposes of this tutorial, we will deploy Stage 1 instead of using existing resources.
* Follow the guide [here], which uses single region scenario.(./terraform_ent_install_aws_stage1.ipynb)
* Here is the source repo for [TFE on AWS Prerequisites](https://github.com/hashicorp-services/terraform-aws-tfe-prereqs)

---

## Provision Stage 2

1. Get repo
1. Set variables

Follow the sections below to deploy TFE for the "happy path" deployment scenario.

### Clone the Repo - Hashi Folks

Create a directory such as `config/terraform/terraform-aws-tfe` into which you want to clone this repository.

In [ ]:
mkdir -p config/terraform/terraform-aws-tfe

Clone the repo.

In [ ]:
git clone https://github.com/hashicorp-services/terraform-aws-tfe config/terraform/terraform-aws-tfe

### unzip Repo - For Non-Hashi Folks

A zip file of the Terraform module `terraform-aws-tfe.zip` is in this repo. You can check with your account team in case there is an updated version.

Extract the archive.

In [ ]:
mkdir -p config/terraform && \
  unzip -qq terraform-aws-tfe.zip -d config/terraform/ && \
  ls config/terraform/terraform-aws-tfe

### Set Main Variables

In [ ]:
export PREFIX=pphan
export TFE_STAGE_1=config/terraform/tfe_stage_1
export ALB_EXT_R53_ACM_OL=config/terraform/terraform-aws-tfe/tests/alb-ext-r53-acm-ol
export AWS_REGION=us-west-2

Create variables of Stage 1 outputs

In [ ]:
export BOOTSTRAP_BUCKET_NAME_PRIMARY=$(terraform -chdir=${TFE_STAGE_1} output -raw bootstrap_bucket_name_primary) && echo $BOOTSTRAP_BUCKET_NAME_PRIMARY
export BASTION_IP=$(terraform -chdir=${TFE_STAGE_1} output -raw bastion_public_ip) && echo $BASTION_IP

Review outputs from Stage 1.

In [ ]:
terraform -chdir=${TFE_STAGE_1} output

### Copy license file to S3 Bootstrap bucket

Place your TFE license file in `${ALB_EXT_R53_ACM_OL}` folder.

Copy it to your S3 bootstrap bucket.

In [ ]:
aws s3 cp ${ALB_EXT_R53_ACM_OL}/tfe-license.rli s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}/

In [ ]:
aws s3 ls s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}

### Create main.tf

In [ ]:
mkdir -p config/terraform/tfe

In [ ]:
cat > ${ALB_EXT_R53_ACM_OL}/versions.tf <<EOF 
terraform {
  required_providers {
    aws      = "~> 3.36.0"
    random   = ">= 2.2.0"
    template = ">= 2.1.2"
  }
}

provider "aws" {
  region = var.aws_region
}
variable "aws_region" {}
EOF

In [ ]:
cat > ${ALB_EXT_R53_ACM_OL}/main.tf <<EOF 
module "tfe" {
  source = "../.."

  friendly_name_prefix = var.friendly_name_prefix
  common_tags          = var.common_tags

  tfe_bootstrap_bucket      = var.tfe_bootstrap_bucket
  tfe_license_filepath      = var.tfe_license_filepath
  tfe_release_sequence      = var.tfe_release_sequence
  tfe_hostname              = var.tfe_hostname
  tfe_install_secrets_arn   = var.tfe_install_secrets_arn
  console_password          = var.console_password
  enc_password              = var.enc_password
  log_forwarding_enabled    = var.log_forwarding_enabled
  cloudwatch_log_group_name = var.cloudwatch_log_group_name

  vpc_id                  = var.vpc_id
  lb_subnet_ids           = var.lb_subnet_ids
  ec2_subnet_ids          = var.ec2_subnet_ids
  rds_subnet_ids          = var.rds_subnet_ids
  ingress_cidr_443_allow  = var.ingress_cidr_443_allow
  ingress_cidr_8800_allow = var.ingress_cidr_8800_allow
  ingress_cidr_22_allow   = var.ingress_cidr_22_allow
  load_balancer_scheme    = var.load_balancer_scheme
  route53_hosted_zone_acm = var.route53_hosted_zone_acm
  route53_hosted_zone_tfe = var.route53_hosted_zone_tfe
  create_tfe_alias_record = var.create_tfe_alias_record

  kms_key_arn = var.kms_key_arn

  asg_instance_count = var.asg_instance_count
  asg_max_size       = var.asg_max_size
  os_distro          = var.os_distro
  ami_id             = var.ami_id
  encrypt_ebs        = var.encrypt_ebs
  ssh_key_pair       = var.ssh_key_pair
  instance_size      = var.instance_size # added

  rds_instance_class      = var.rds_instance_class # added
  rds_password            = var.rds_password
  rds_multi_az            = var.rds_multi_az
  rds_skip_final_snapshot = var.rds_skip_final_snapshot

  enable_active_active             = var.enable_active_active
  redis_subnet_ids                 = var.redis_subnet_ids
  redis_password                   = var.redis_password
  redis_at_rest_encryption_enabled = var.redis_at_rest_encryption_enabled
}
EOF

### Create outputs.tf

We will using the existing `outputs.tf` file. Run this command if you want to view it.

In [ ]:
cat ${ALB_EXT_R53_ACM_OL}/outputs.tf

Alternatively, you can use this if you are on a Mac with vscode. 

In [ ]:
code ${ALB_EXT_R53_ACM_OL}/outputs.tf

Create a new file to get more outputs out of terraform. TO BE COMPLETED

In [ ]:
cat ${ALB_EXT_R53_ACM_OL}/outputs_new.tf <<EOF
output "tfe_rds" {
  value = module.tfe.RDS
}
EOF

### Create variables.tf

We will using the existing `variables.tf` file. Run this command if you want to view it.

In [ ]:
cat ${ALB_EXT_R53_ACM_OL}/variables.tf

Alternatively, you can use this if you are on a Mac with vscode. 

In [ ]:
code ${ALB_EXT_R53_ACM_OL}/variables.tf

### Create tfvars file to set variables.

#### Common Variables
<details><summary></summary>

</details>
Here are the values you should set in the tfvars section below.

* `friendly_name_prefix` - the same namespace you set in [Stage 1](bear://x-callback-url/open-note?id=BE232B3D-94AE-49B6-9B33-B62095551C8C-66557-000513BBCB51AF13&header=Set%20Terraform%20Variables). ex: `"pphan"`
* `aws_region` - setting from env var, but you can change
* `common_tags` - customize as needed
	* `Owner` and `TTL` are used within HashiCorp's own AWS account for resource reaping purposes.

```go
common_tags = {
  Owner       = "pphan"
  Environment = "Test"
  Tool        = "Terraform"
  TTL         = "1day"
}
```


#### TFE Variables

Here are the values you should set in the tfvars section below.

* `friendly_name_prefix` - your resource names will be prefixed with this
* `tfe_hostname` - ex "`pphan-tfe.hashidemos.io`"
* `tfe_release_sequence` - `0` for the latest; specify a number to pin it to that [release](https://www.terraform.io/docs/enterprise/release/index.html)
* `tfe_license_file_path` - must use s3 bootstrap bucket for this module


<details><summary>Compute Variables</summary>

</details>

* Set `instance_size` to
	* `m5.xlarge` for demos and POCs
	* "`m5.2xlarge`" or larger for production
        * rule of thumb is 2GB memory per run, 32GB memory will support about 14 concurrent runs. 4GB for TFE.
    * for larger deployments consider the `r5.2xlarge` instead; more memory allows you to support more concurrent runs
* Set `os_distro` to
	* `ubuntu`, "amzn2", "rhel", "centos"; default is `amzn2`

#### Database Variables
<details><summary></summary>
    
</details>

* Set `rds_instance_class`:
	* "`db.m5.xlarge`" for Demos, POCs (default)
	* "`db.m5.xlarge`" or "`db.m5.2xlarge`" for production.
* Set `rds_multi_az` to
	* "`false`" for Demos
	* "`true`" for POCs and Production (default)

#### Network Variables
<details><summary></summary>

</details>

* Set `vpc_id`, `ec2_subnet_ids`, `rds_subnet_ids`, and `lb_subnet_ids` to the corresponding **outputs** from Stage 1 or the IDs of the resources you created using other means.
```go
vpc_id                    = "vpc-07b89518584abd90b"
lb_subnet_ids             = ["subnet-04be8d0fe8aba96a4", "subnet-0b623ade19e6271c5"] 
ec2_subnet_ids            = ["subnet-04be8d0fe8aba96a4", "subnet-0b623ade19e6271c5"] 
rds_subnet_ids            = ["subnet-04be8d0fe8aba96a4", "subnet-0b623ade19e6271c5"] 
```
    * We pull the values from env var's, terraform output, etc when possible.
* Note on Subnets
	* The `*_subnet_ids` should be in the form `"[<subnet_1>","<subnet_2>"]`.
	* The `ec2` and `rds` subnets can be distinct or the same and can be public or private.
	* The `alb` subnets must be **public**.
* Set `route53_hosted_zone` - ex `hashidemos.io` 
* Set `kms_key_arn` to the `kms_key_arn` output from [Stage 1](./TerraformInstallStage1.ipynb) or the ID of the KMS key you created using other means.
* Access to Console Port (8800)
    * I added my machine's IP to the LB SG allow for 8800, so that I can connect to it directly.
    * Adjust accordingly for your environment
    * `ingress_cidr_8800_allow = ["10.0.0.0/16","$(curl -s http://ipv4.icanhazip.com)/32"]`

<br>

Set the rest of the variables in the `<linux_flavor>.auto.tfvars` file.


* Be sure to adjust the following variables if deploying for a POC or in production.
    * `aws_instance_type`
    * `database_storage`
    * `database_instance_class`
    * `database_multi_az`


* Set `ssh_key_pair` to the name of your SSH keypair as it is displayed in the AWS Console.
	* `ssh_key_pair = "pphan-ptfe-ec2-key"`
	* [::pp::] I added a resource to create a `aws_key_pair` but you might already have one. Else you can manually create one in AWS console.
    
```go
resource "aws_key_pair" "ec2_key" {
  key_name   = "${var.namespace}-ec2-key"
  public_key = "<key>"
}
```

#### New tfvars

This module currently does not allow you to specify the volume type, which is currently set to `gp2`. We recommend you change this to `gp3` and increase the volume size from `50` to `100`. You can change manually change these values in `compute.tf`.

> `gp3` volumes provide more IOPS than `gp2` volumes. Learn more [here](https://aws.amazon.com/about-aws/whats-new/2020/12/introducing-new-amazon-ebs-general-purpose-volumes-gp3/)

Alternatively, this `sed` command can change it for you.

In [ ]:
sed -i '' -e 's/volume_type =.*/volume_type = "gp3"/' \
  -e 's/volume_size =.*/volume_size = 100/' \
  ${ALB_EXT_R53_ACM_OL}/../../compute.tf

Copy license file - NOT NEEDED; WILL COPY TO S3

In [ ]:
cp tfe-license.rli ${ALB_EXT_R53_ACM_OL}

We will set required and optional variables. Some of the values are pulled from our stage 1 deployment. Replace these with your own values if you didn't use the Stage 1 process.

In [ ]:
tee ${ALB_EXT_R53_ACM_OL}/alb-ext-r53-acm-ol-ubuntu.auto.tfvars <<EOF
#------------------------------------------------------------------------------
# COMMON
#------------------------------------------------------------------------------
friendly_name_prefix = "${PREFIX}"
aws_region    = "${AWS_REGION}"
common_tags   = {
  "App"       = "TFE"
  "Env"       = "test"
  "Scenario"  = "alb-ext-r53-acm-ol"
  "Terraform" = "local-cli"
  "Owner"     = "YourName"
  "TTL"       = "1day"
}

#------------------------------------------------------------------------------
# TFE
#------------------------------------------------------------------------------
tfe_bootstrap_bucket      = $(terraform -chdir=${TFE_STAGE_1} output bootstrap_bucket_name_primary)
tfe_license_filepath      = "s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}/tfe-license.rli"
tfe_release_sequence      = 0
tfe_hostname              = "pphan-tfe.hashidemos.io"
console_password          = "pphan_aws_secretsmanager" #"aws_secretsmanager"
enc_password              = "pphan_aws_secretsmanager" #"aws_secretsmanager"
tfe_install_secrets_arn   = $(terraform -chdir=${TFE_STAGE_1} output secretsmanager_secret_metadata_arn)
log_forwarding_enabled    = true
cloudwatch_log_group_name = "tfe-log-group"

#------------------------------------------------------------------------------
# NETWORK - data needs to be refreshed everytime - from Stage 1 outputs
#------------------------------------------------------------------------------
vpc_id                  = $(terraform -chdir=${TFE_STAGE_1} output vpc)
lb_subnet_ids           = $(terraform -chdir=${TFE_STAGE_1} output public_subnet_ids)
ec2_subnet_ids          = $(terraform -chdir=${TFE_STAGE_1} output private_subnet_ids)
rds_subnet_ids          = $(terraform -chdir=${TFE_STAGE_1} output private_subnet_ids)
ingress_cidr_443_allow  = ["0.0.0.0/0"]
ingress_cidr_8800_allow = ["10.0.0.0/16","$(curl -s http://ipv4.icanhazip.com)/32"]
ingress_cidr_22_allow   = ["10.0.0.0/16"]
load_balancer_scheme    = "external"
route53_hosted_zone_acm = "hashidemos.io"
create_tfe_alias_record = true
route53_hosted_zone_tfe = "hashidemos.io"

kms_key_arn = $(terraform -chdir=${TFE_STAGE_1} output kms_key_arn)

#------------------------------------------------------------------------------
# COMPUTE
#------------------------------------------------------------------------------
asg_instance_count = 1
asg_max_size       = 1
os_distro          = "ubuntu" #amzn2
ssh_key_pair       = $(terraform -chdir=${TFE_STAGE_1} output ssh_key_pair)
instance_size      = "m5.xlarge" 

#------------------------------------------------------------------------------
# DATABASE
#------------------------------------------------------------------------------
rds_instance_class = "db.m5.xlarge"
rds_username       = "tfe" # Default is tfe
rds_password       = "MyRdsPassword123!"
rds_multi_az       = true

# Active/Active
enable_active_active             = false
redis_subnet_ids                 = $(terraform -chdir=${TFE_STAGE_1} output private_subnet_ids)
redis_password                   = "MyRedisPassword123!"
redis_at_rest_encryption_enabled = true

#------------------------------------------------------------------------------
# STORAGE
#------------------------------------------------------------------------------
# None needed
EOF

### Set Credentials

Set AWS Credentials as environment variables.

In [ ]:
export AWS_ACCESS_KEY_ID=REPLACE_ME
export AWS_SECRET_ACCESS_KEY=REPLACE_ME

**FOR HASHI ONLY:** Use `doormat` to get AWS Credentials

In [ ]:
awscredsenv

Verify credentials were set. Using `envo` if available.

In [ ]:
# Check to see if a variable is set, but don't show values
envo | grep AWS || env | grep AWS | cut -d= -f1

### Provision TFE - Initialize and Apply

Initialize the **Stage 2** Terraform configuration and download providers.

In [ ]:
terraform -chdir=${ALB_EXT_R53_ACM_OL} init

In [ ]:
terraform -chdir=${ALB_EXT_R53_ACM_OL} validate

In [ ]:
terraform -chdir=${ALB_EXT_R53_ACM_OL} plan -input=false

Provision the Stage 2 resources.

In [ ]:
terraform -chdir=${ALB_EXT_R53_ACM_OL} apply -input=false -auto-approve \
  > /tmp/tf_stage_2_apply.out 2>&1

Output is sent to `/tmp/tf_stage_2_apply.out`. View that file from a terminal to check progress/results.
```shell
tail -f /tmp/tf_stage_2_apply.out
```

> NOTE: This takes approximately 15 minutes (14 just for RDS)

Sample Outputs:
```shell
Outputs:

tfe_admin_console_url = "https://pphan-tfe.hashidemos.io:8800"
tfe_lb_dns_name = "pphan-tfe-web-alb-664253854.us-west-2.elb.amazonaws.com"
tfe_s3_bucket_name = "pphan-tfe-app-us-west-2-<aws_acct_id>"
tfe_url = "https://pphan-tfe.hashidemos.io"
```

## Monitoring and Verifying the Deployment

The following steps are not necessary for setup, but useful for verifying the deployment.

### SSH to EC2 instance

#### Get IP of TFE Instance

1. Wait until you see outputs for the `apply`.
1. Go to the **AWS Console > EC2 > Auto Scaling Groups**.
1. Find your ASG. You can filter by your `prefix` to narrow down the results.
1. Select your ASG.
	* My ASG is called: `pphan-tfe-asg`
1. Click **Instance management** tab.
1. Click the Instance ID link.
1. Copy the Private IP address.

1. Click the **Connect** button and copy the SSH connection command.
1. Type that command in a shell that contains your SSH private key from your AWS key pair and connect to your primary PTFE instance. (It might not be ready right away.)

The TFE instance is placed in a private subnet and has no public IP. To connect to it you will need to go through a bastion host. This is covered next.

In [ ]:
ASG_NAME=$(jq -r '.resources[].instances[].attributes.name' \
  ${ALB_EXT_R53_ACM_OL}/terraform.tfstate \
  | grep tfe-asg) && echo $ASG_NAME

Grab the instance ID from ASG, get private IP address of instance, and store value into `TFE_PRIVATE_IP`.

In [ ]:
TFE_PRIVATE_IP=$(aws ec2 describe-instances --instance-ids i-0d707c5706eee4b6d \
  $(aws autoscaling describe-auto-scaling-instances --output text \
  --query "AutoScalingInstances[?AutoScalingGroupName=='${ASG_NAME}'].InstanceId") \
  --query "Reservations[].Instances[].PrivateIpAddress" --output text) && $TFE_PRIVATE_IP

#### Connect to Bastion

To ssh to the TFE instance, I use the bastion host deployed in Stage 1. You can't run interactive commands in Jupyter Notebooks, so you use the following command to see what you should run from a CLI terminal. NOTE: You might need to change the username based on `os_distro`.

First hop. Connect to Bastion.

In [ ]:
ssh -A ec2-user@${BASTION_IP} \
  ssh -A -o StrictHostKeyChecking=no ubuntu@${TFE_PRIVATE_IP} \
  tail /var/log/cloud-init-output.log

Sample output
```shell
ssh -A ec2-user@34.210.115.80
```

Then, from the bastion host I run this command. Replace IP with your specific one.
```shell
ssh -A ubuntu@<tfe_instance_private_ip>
```

> My bastion host defaulted to `amzn2` but for TFE i selected `ubuntu`.

More info on [transparent multihop ssh](http://sshmenu.sourceforge.net/articles/transparent-mulithop.html).

Quick and Dirty Hack: You can chain ssh commands together and/or create an alias to save on typing. This also allows us to run them in Jupyter since it avoids interacting with the terminal.

```shell
ssh -A ec2-user@34.210.115.80 \
    ssh -A ubuntu@10.0.254.113 tail /var/log/cloud-init-output.log
```

### Monitor the deployment

To monitor the progress of the install (cloud-init process):

1. SSH into the EC2 instance
2. run `journalctl -xu cloud-final -f` to tail the logs (or remove the `-f` if the cloud-init process has finished).
3. If the operating system is Ubuntu, logs can also be viewed via:
    * `tail /var/log/cloud-init-output.log`
    * add `-f` to follow it
    * NOTE: It is **ok** if you see multiple warnings in the log like:
```shell
curl: (6) Could not resolve host: <ptfe_dns>
```
	* This means that the script has run the installer and is currently testing the availability of the TFE application with `curl` every **5 seconds**.
	* You will also see this warning.
```shell
curl: (22) The requested URL returned error: 502 Bad Gateway
```
	* If this lasts for more than **5 minutes**, then something is wrong.
	* When the `/var/log/cloud-init-output.log` stops showing `curl` calls against the hostname and instead shows `INFO: TFE user_data script finished successfully!` then things should be good.

* The install script dumps the out to install-ptfe.log. You can view or tail it to see the bootstrap logs and `curl` test against TFE. 
    * `tail -f install-ptfe.log`
    * `less install-ptfe.log`
    * NOTE: This is based on Roger's [install script](https://github.com/hashicorp/private-terraform-enterprise/blob/automated-aws-pes-installation/examples/aws/user-data-ubuntu-online.tpl) and not available here.

* You can tail the audit log and watch it as you click around the TFE UI.  Just log into your server and run
    * `docker logs -f ptfe_atlas | grep "Audit Log"`
    * `docker logs ptfe_atlas --since 5m | grep "Audit Log"`

``` shell
+ curl -ksfS --connect-timeout 5 https://10.110.1.46/_health_check
OK+ '[' true == true ']'
+ echo 'Creating initial admin user and organization'
Creating initial admin user and organization
+ cat
++ replicated admin --tty=0 retrieve-iact
+ initial_token=OtzRwvj4yNNPSXfP9odg23JFcPtra84t
```

### Check postgres

This is an optional check. It makes sure your DB was installed and the database configured.

Get DB identifier.

1. Go to this page.

In [ ]:
open https://${AWS_REGION}.console.aws.amazon.com/rds/home?region=${AWS_REGION}#databases:

2. Filter by your `prefix`.
3. Click on your DB.
4. Copy your Endpoint address.<br> ex `pphan-tfe-rds-<aws_acct_id>.cyqfwyfsxjlb.us-west-2.rds.amazonaws.com`

Or you can use this.

In [ ]:
DB_ADDRESS=$(jq -r '.resources[].instances[].attributes.address' \
  ${ALB_EXT_R53_ACM_OL}/terraform.tfstate \
  | grep rds) && echo $DB_ADDRESS

5. From your bastion session, connect with `psql`. Replace the endpoint and password with your own.

In [ ]:
ssh -A ec2-user@${BASTION_IP} \
  ssh -A -o StrictHostKeyChecking=no ubuntu@${TFE_PRIVATE_IP} /bin/bash <<EOF
    sudo DEBIAN_FRONTEND=noninteractive apt-get install -qq postgresql-client > /dev/null
    PGPASSWORD="MyRdsPassword123!" psql \
    -h $DB_ADDRESS -d tfe -U tfe -c "\dn"
EOF  

* `-h` is host. Copied from above.
* `-d` is database name. Default is `tfe`
* `-U` is username. Default is `tfe`
* You will provide password via env var. See `rds_password`.

Sample Output
```
 List of schemas
   Name   | Owner 
----------+----------
 public   | tfe
 rails    | tfe
 registry | tfe
 vault    | tfe
(4 rows)
```

## Log In to TFE

After the module has been deployed and the EC2 instance created by the Autoscaling Group has finished initializing, open a web browser and log in to the new TFE instance.

1. Give the EC2 instance around 7-12 minutes after it has finished initializing before launching a browser and attempting to login to TFE.
1. Connect to the TFE Admin Console on port 8800 (`https://<tfe_hostname>:8800`)
    * This is the load balancer URL.

Open Admin Console

In [ ]:
open $(terraform -chdir=${ALB_EXT_R53_ACM_OL} output -raw tfe_url):8800

3. Authenticate using the value of `console_password`.
    * See `tfvars` file.
1. From the Dashboard page, after verifying the application is started, click the "**Open**" link underneath the **Stop Now** button.
1. Create the Initial Admin User 
    * Username, Email, Password
    * Click `Create an account`.
1. Create the Initial Organization.
    * Organization name, Email address
    * Click `Create organization`.
1. Create a new Workspace or skip to other Post-Deployment steps.
<p>&nbsp;</p>

* The username defaults to `admin` unless a value was specified for the input `tfe_initial_admin_username` input. 
* The **password** is the value set for the `tfe_initial_admin_pw` variable.
	* If not set, then it will be `random_password.tfe_initial_admin_pw.result` (found in the Terraform State file).
* The **console password** can be found here `terraform.tfstate > console_password`.

Open UI

In [ ]:
open $(terraform -chdir=${ALB_EXT_R53_ACM_OL} output -raw tfe_url)

---

## Repaving Your PTFE Instances With Terraform

* You can replace or "repave" the EC2 instance(s) running TFE with Terraform at any time by following this process:
	* Terminate the EC2 instance(s) in the AWS Console
* This will cause the EC2 instance(s) to be destroyed and recreated.
* In addition, it will cause the `aws_lb_target_group_attachment` resources associated with the application load balancer to be destroyed and recreated.
* This ensures that the ALB will always point to the primary TFE instance.

---

## Resources

## Appendix

### Creating Initial User and Inital Org vi API

Set variables

In [ ]:
tfe_initial_admin_username=admin
tfe_initial_admin_email=pphan@hashicorp.com
tfe_initial_admin_pw=TVCN4life!

tfe_hostname=https://pphan-tfe.hashidemos.io
tfe_config_dir=/tmp
tfe_initial_org_name=pphan_corp
tfe_initial_org_email=$tfe_initial_admin_email

printf "#==> Make sure variables are set.
$tfe_initial_admin_username
$tfe_initial_admin_email
$tfe_initial_admin_pw
$tfe_hostname
$tfe_initial_org_name
$tfe_initial_org_email
"

Create initial user

In [ ]:
# build payload for initial TFE admin user
tee $tfe_config_dir/initial_admin_user.json <<EOF
{
	"username": "${tfe_initial_admin_username}",
	"email": "${tfe_initial_admin_email}",
	"password": "${tfe_initial_admin_pw}"
}
EOF

# retrieve Initial Admin Creation Token
iact=$(ssh -A ec2-user@34.210.115.80 \
    ssh -A ubuntu@10.0.254.113 \
    replicated admin --tty=0 retrieve-iact)

# HTTP POST to retrieve Initial Admin User Token
iaut=$(curl --header "Content-Type: application/json" --request POST --data @$tfe_config_dir/initial_admin_user.json "${tfe_hostname}/admin/initial-admin-user?token=$iact" | jq -r '.token')
echo $iaut

Create initial organization

In [ ]:
# build payload for initial TFE Organization creation
tee $tfe_config_dir/initial_org.json <<EOF
{
  "data": {
    "type": "organizations",
    "attributes": {
      "name": "${tfe_initial_org_name}",
      "email": "${tfe_initial_org_email}"
    }
  }
}
EOF

# HTTP POST to create initial TFE Organization
curl -H "Authorization: Bearer $iaut" \
  -H "Content-Type: application/vnd.api+json" \
  --request POST --data @$tfe_config_dir/initial_org.json \
  "${tfe_hostname}/api/v2/organizations" | jq -c .data

### Fun Hacks

#### Getting additional data from state file

In [ ]:
jq -r '.resources[].instances[].attributes.address' \
  ${ALB_EXT_R53_ACM_OL}/terraform.tfstate \
  | grep rds

---

## Cleanup

### Destroy TFE

Destroy TFE. This will take about 16 minutes. It will error out due to a lingering DB snapshot. The error message will provide the name of the snapshot to delete.

In [ ]:
time terraform -chdir=${ALB_EXT_R53_ACM_OL} destroy -input=false -auto-approve > /tmp/tf_stage_2_destroy.out 2>&1

Expected Output
```
│ Error: error deleting Database Instance "pphan-tfe-rds-<aws_acct_id>": DBSnapshotAlreadyExists: Cannot create the snapshot because a snapshot with the identifier pphan-tfe-rds-<aws_acct_id>-final-snapshot already exists.
```

Delete DB snapshot. Be sure to replace identifier with your own value.

In [ ]:
aws rds delete-db-snapshot \
  --db-snapshot-identifier pphan-tfe-rds-<aws_acct_id>-final-snapshot

Run `terraform destroy` again.

In [ ]:
time terraform -chdir=${ALB_EXT_R53_ACM_OL} destroy -input=false -auto-approve >> /tmp/tf_stage_2_destroy.out 2>&1

### Destroy artifacts

Delete json data files.

In [ ]:
rm -rf /tmp/initial_admin_user.json /tmp/initial_org.json

Delete terraform ouput files

In [ ]:
rm -rf /tmp/tf_stage_2_*.out

In [ ]:
#debugging
rm -rf config/terraform/tfe/cloud_init_deleteme

Delete license file from S3 bucket.

In [ ]:
aws s3 rm s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}/tfe-license.rli

In [ ]:
aws s3 ls s3://${BOOTSTRAP_BUCKET_NAME_PRIMARY}

## Changes
* Add a resource to create a `aws_key_pair` in `custom.tf` in root module.

```shell
resource "aws_key_pair" "ec2_key" {
  key_name   = "${var.namespace}-ec2-key"
  public_key = "<pub_key>"
}
```

* Add the following to `database/main.tf` resource `aws_db_instance.ptfe`.
    * This allows me to easily destroy the environment. Do **NOT** do this in production.

```shell
  # changes
  skip_final_snapshot       = "true"
  backup_retention_period   = "0" # disable backup
```

**END**